In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix

In [38]:
B = pd.read_csv('bank.csv')
B.head()

,age,job,marital,education,default,balance,housing-loan,personal-loan,current-campaign,previous-campaign,subscribed
0,30,unemployed,married,primary,no,1787,no,no,1,0,no
1,33,services,married,secondary,no,4789,yes,yes,1,4,no
2,35,management,single,tertiary,no,1350,yes,no,1,1,no
3,30,management,married,tertiary,no,1476,yes,yes,4,0,no
4,59,blue-collar,married,secondary,no,0,yes,no,1,0,no


In [39]:
B.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4521 entries, 0 to 4520
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   age                4521 non-null   int64 
 1   job                4521 non-null   object
 2   marital            4521 non-null   object
 3   education          4521 non-null   object
 4   default            4521 non-null   object
 5   balance            4521 non-null   int64 
 6   housing-loan       4521 non-null   object
 7   personal-loan      4521 non-null   object
 8   current-campaign   4521 non-null   int64 
 9   previous-campaign  4521 non-null   int64 
 10  subscribed         4521 non-null   object
dtypes: int64(4), object(7)
memory usage: 388.6+ KB


In [40]:
B['subscribed'].value_counts()

no     4000
yes     521
Name: subscribed, dtype: int64

In [41]:
from sklearn.utils import resample

In [42]:
bsn = B[B['subscribed']=='no']
bsy = B[B['subscribed'] =='yes']

In [43]:
BMU = resample(bsy, replace=True, n_samples=2000, random_state=42)   # bank minority unsampled

In [44]:
NB = pd.concat([bsn, BMU])         # new bank
NB['subscribed'].value_counts()


no     4000
yes    2000
Name: subscribed, dtype: int64

In [45]:
# NB = shuffle(NB)

In [46]:
Xf = list(NB.columns)
Xf.remove('subscribed')
Xf

['age',
 'job',
 'marital',
 'education',
 'default',
 'balance',
 'housing-loan',
 'personal-loan',
 'current-campaign',
 'previous-campaign']

In [47]:
EB = pd.get_dummies(NB[Xf], drop_first=True)
X = EB
X

,age,balance,current-campaign,previous-campaign,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,...,job_unemployed,job_unknown,marital_married,marital_single,education_secondary,education_tertiary,education_unknown,default_yes,housing-loan_yes,personal-loan_yes
0,30,1787,1,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,0
1,33,4789,1,4,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,1,1
2,35,1350,1,1,0,0,0,1,0,0,...,0,0,0,1,0,1,0,0,1,0
3,30,1476,4,0,0,0,0,1,0,0,...,0,0,1,0,0,1,0,0,1,1
4,59,0,1,0,1,0,0,0,0,0,...,0,0,1,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
619,35,7050,3,4,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
1177,28,4579,2,0,0,0,0,0,0,1,...,0,0,0,1,0,1,0,0,0,0
3498,58,462,1,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4366,59,0,1,0,0,0,0,0,1,0,...,0,0,1,0,1,0,0,0,0,0


In [48]:
Y = NB['subscribed'].map(lambda x: int(x=='yes'))

In [49]:
Cmat = X.corr().abs()

In [50]:
Cmat

,age,balance,current-campaign,previous-campaign,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,...,job_unemployed,job_unknown,marital_married,marital_single,education_secondary,education_tertiary,education_unknown,default_yes,housing-loan_yes,personal-loan_yes
age,1.000000,0.116661,0.017923,0.001506,0.066583,0.004310,0.082258,0.067890,0.525063,0.006086,...,0.004726,0.078146,0.273028,0.423815,0.098799,0.113228,0.094899,0.008845,0.194625,0.026840
balance,0.116661,1.000000,0.012778,0.030651,0.066231,0.000353,0.058340,0.040634,0.097046,0.005872,...,0.017251,0.001199,0.026961,0.008208,0.082181,0.073698,0.029869,0.072134,0.045138,0.072443
current-campaign,0.017923,0.012778,1.000000,0.088047,0.018551,0.015274,0.012967,0.036582,0.046458,0.042065,...,0.015985,0.016831,0.017385,0.010400,0.022921,0.023740,0.011159,0.014455,0.012254,0.041865
previous-campaign,0.001506,0.030651,0.088047,1.000000,0.030009,0.027656,0.031959,0.004984,0.011355,0.020565,...,0.019691,0.003907,0.014900,0.041845,0.030310,0.014123,0.018997,0.036346,0.002664,0.045438
job_blue-collar,0.066583,0.066231,0.018551,0.030009,1.000000,0.092666,0.077608,0.257391,0.129608,0.098380,...,0.081728,0.050548,0.092239,0.073857,0.062857,0.306804,0.006547,0.001397,0.202778,0.032115
job_entrepreneur,0.004310,0.000353,0.015274,0.027656,0.092666,1.000000,0.030826,0.102235,0.051480,0.039076,...,0.032462,0.020077,0.058772,0.057149,0.045030,0.033271,0.024552,0.049362,0.023786,0.043399
job_housemaid,0.082258,0.058340,0.012967,0.031959,0.077608,0.030826,1.000000,0.085622,0.043114,0.032726,...,0.027187,0.016815,0.028146,0.050178,0.074172,0.050027,0.006021,0.023249,0.083101,0.010175
job_management,0.067890,0.040634,0.036582,0.004984,0.257391,0.102235,0.085622,1.000000,0.142992,0.108539,...,0.090167,0.055768,0.024646,0.030925,0.409287,0.580526,0.038183,0.020744,0.035294,0.035393
job_retired,0.525063,0.097046,0.046458,0.011355,0.129608,0.051480,0.043114,0.142992,1.000000,0.054654,...,0.045403,0.028082,0.056648,0.138788,0.028552,0.074505,0.012425,0.017228,0.179368,0.025532
job_self-employed,0.006086,0.005872,0.042065,0.020565,0.098380,0.039076,0.032726,0.108539,0.054654,1.000000,...,0.034464,0.021316,0.011318,0.000530,0.071686,0.110664,0.001386,0.002326,0.041987,0.000165


In [51]:
upper = Cmat.where(np.triu(np.ones(Cmat.shape),k=1).astype(bool))
upper

,age,balance,current-campaign,previous-campaign,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,...,job_unemployed,job_unknown,marital_married,marital_single,education_secondary,education_tertiary,education_unknown,default_yes,housing-loan_yes,personal-loan_yes
age,NaN,0.116661,0.017923,0.001506,0.066583,0.004310,0.082258,0.067890,0.525063,0.006086,...,0.004726,0.078146,0.273028,0.423815,0.098799,0.113228,0.094899,0.008845,0.194625,0.026840
balance,NaN,NaN,0.012778,0.030651,0.066231,0.000353,0.058340,0.040634,0.097046,0.005872,...,0.017251,0.001199,0.026961,0.008208,0.082181,0.073698,0.029869,0.072134,0.045138,0.072443
current-campaign,NaN,NaN,NaN,0.088047,0.018551,0.015274,0.012967,0.036582,0.046458,0.042065,...,0.015985,0.016831,0.017385,0.010400,0.022921,0.023740,0.011159,0.014455,0.012254,0.041865
previous-campaign,NaN,NaN,NaN,NaN,0.030009,0.027656,0.031959,0.004984,0.011355,0.020565,...,0.019691,0.003907,0.014900,0.041845,0.030310,0.014123,0.018997,0.036346,0.002664,0.045438
job_blue-collar,NaN,NaN,NaN,NaN,NaN,0.092666,0.077608,0.257391,0.129608,0.098380,...,0.081728,0.050548,0.092239,0.073857,0.062857,0.306804,0.006547,0.001397,0.202778,0.032115
job_entrepreneur,NaN,NaN,NaN,NaN,NaN,NaN,0.030826,0.102235,0.051480,0.039076,...,0.032462,0.020077,0.058772,0.057149,0.045030,0.033271,0.024552,0.049362,0.023786,0.043399
job_housemaid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.085622,0.043114,0.032726,...,0.027187,0.016815,0.028146,0.050178,0.074172,0.050027,0.006021,0.023249,0.083101,0.010175
job_management,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.142992,0.108539,...,0.090167,0.055768,0.024646,0.030925,0.409287,0.580526,0.038183,0.020744,0.035294,0.035393
job_retired,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.054654,...,0.045403,0.028082,0.056648,0.138788,0.028552,0.074505,0.012425,0.017228,0.179368,0.025532
job_self-employed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.034464,0.021316,0.011318,0.000530,0.071686,0.110664,0.001386,0.002326,0.041987,0.000165


In [52]:
Drop = [column for column in upper.columns if any(upper[column]>0.80)]
Drop

[]

In [53]:
X.drop(Drop, axis=1, inplace=True)
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6000 entries, 0 to 4328
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype
---  ------               --------------  -----
 0   age                  6000 non-null   int64
 1   balance              6000 non-null   int64
 2   current-campaign     6000 non-null   int64
 3   previous-campaign    6000 non-null   int64
 4   job_blue-collar      6000 non-null   uint8
 5   job_entrepreneur     6000 non-null   uint8
 6   job_housemaid        6000 non-null   uint8
 7   job_management       6000 non-null   uint8
 8   job_retired          6000 non-null   uint8
 9   job_self-employed    6000 non-null   uint8
 10  job_services         6000 non-null   uint8
 11  job_student          6000 non-null   uint8
 12  job_technician       6000 non-null   uint8
 13  job_unemployed       6000 non-null   uint8
 14  job_unknown          6000 non-null   uint8
 15  marital_married      6000 non-null   uint8
 16  marital_single       600

### IPL Dataset

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

In [19]:
I = pd.read_excel('IPL Raw Data.xlsx')
I

,Sl.NO.,PLAYER NAME,AGE,COUNTRY,TEAM,PLAYING ROLE,T-RUNS,T-WKTS,ODI-RUNS-S,ODI-SR-B,...,SR -B,SIXERS,RUNS-C,WKTS,AVE-BL,ECON,SR-BL,AUCTION YEAR,BASE PRICE,SOLD PRICE
0,1,"Abdulla, YA",2,SA,KXIP,Allrounder,0,0,0,0.00,...,0.000000,0,307,15,20.466667,8.898551,13.933333,2009,50000,50000
1,2,Abdur Razzak,2,BAN,RCB,Bowler,214,18,657,71.41,...,0.000000,0,29,0,0.000000,14.500000,0.000000,2008,50000,50000
2,3,"Agarkar, AB",2,IND,KKR,Bowler,571,58,1269,80.62,...,121.014493,5,1059,29,36.517241,8.810316,24.896552,2008,200000,350000
3,4,"Ashwin, R",1,IND,CSK,Bowler,284,31,241,84.56,...,76.315789,0,1125,49,22.959184,6.232687,22.142857,2011,100000,850000
4,5,"Badrinath, S",2,IND,CSK,Batsman,63,0,79,45.93,...,120.714940,28,0,0,0.000000,0.000000,0.000000,2011,100000,800000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,126,"Yadav, AS",2,IND,DC,Batsman,0,0,0,0.00,...,125.640000,2,0,0,0.000000,0.000000,0.000000,2010,50000,750000
126,127,Younis Khan,2,PAK,RR,Batsman,6398,7,6814,75.78,...,42.850000,0,0,0,0.000000,0.000000,0.000000,2008,225000,225000
127,128,Yuvraj Singh,2,IND,KXIP+,Batsman,1775,9,8051,87.58,...,131.876333,67,569,23,24.739130,7.024691,21.130435,2011,400000,1800000
128,129,Zaheer Khan,2,IND,MI+,Bowler,1114,288,790,73.55,...,91.666667,1,1783,65,27.430769,7.745439,21.261538,2008,200000,450000


In [20]:
I.drop(['Sl.NO.','PLAYER NAME','TEAM'], axis=1 , inplace = True)

In [21]:
I.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130 entries, 0 to 129
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   AGE            130 non-null    int64  
 1   COUNTRY        130 non-null    object 
 2   PLAYING ROLE   130 non-null    object 
 3   T-RUNS         130 non-null    int64  
 4   T-WKTS         130 non-null    int64  
 5   ODI-RUNS-S     130 non-null    int64  
 6   ODI-SR-B       130 non-null    float64
 7   ODI-WKTS       130 non-null    int64  
 8   ODI-SR-BL      130 non-null    float64
 9   CAPTAINCY EXP  130 non-null    int64  
 10  RUNS-S         130 non-null    int64  
 11  HS             130 non-null    int64  
 12  AVE            130 non-null    float64
 13  SR -B          130 non-null    float64
 14  SIXERS         130 non-null    int64  
 15  RUNS-C         130 non-null    int64  
 16  WKTS           130 non-null    int64  
 17  AVE-BL         130 non-null    float64
 18  ECON      

In [22]:
Xf=I.columns[:-3]
Xf

Index(['AGE', 'COUNTRY', 'PLAYING ROLE', 'T-RUNS', 'T-WKTS', 'ODI-RUNS-S',
       'ODI-SR-B', 'ODI-WKTS', 'ODI-SR-BL', 'CAPTAINCY EXP', 'RUNS-S', 'HS ',
       'AVE', 'SR -B', 'SIXERS', 'RUNS-C', 'WKTS', 'AVE-BL', 'ECON', 'SR-BL'],
      dtype='object')

In [23]:
I['PLAYING ROLE'].unique()

array(['Allrounder', 'Bowler', 'Batsman', 'W. Keeper'], dtype=object)

In [24]:
pd.get_dummies(I['PLAYING ROLE'])

,Allrounder,Batsman,Bowler,W. Keeper
0,1,0,0,0
1,0,0,1,0
2,0,0,1,0
3,0,0,1,0
4,0,1,0,0
...,...,...,...,...
125,0,1,0,0
126,0,1,0,0
127,0,1,0,0
128,0,0,1,0


In [25]:
Cf = ['AGE', 'COUNTRY', 'PLAYING ROLE', 'CAPTAINCY EXP']

In [26]:
In=pd.get_dummies(I[Xf], columns= Cf, drop_first=True)

In [27]:
In.columns

Index(['T-RUNS', 'T-WKTS', 'ODI-RUNS-S', 'ODI-SR-B', 'ODI-WKTS', 'ODI-SR-BL',
       'RUNS-S', 'HS ', 'AVE', 'SR -B', 'SIXERS', 'RUNS-C', 'WKTS', 'AVE-BL',
       'ECON', 'SR-BL', 'AGE_2', 'AGE_3', 'COUNTRY_BAN', 'COUNTRY_ENG',
       'COUNTRY_IND', 'COUNTRY_NZ', 'COUNTRY_PAK', 'COUNTRY_SA', 'COUNTRY_SL',
       'COUNTRY_WI', 'COUNTRY_ZIM', 'PLAYING ROLE_Batsman',
       'PLAYING ROLE_Bowler', 'PLAYING ROLE_W. Keeper', 'CAPTAINCY EXP_1'],
      dtype='object')

In [28]:
In

,T-RUNS,T-WKTS,ODI-RUNS-S,ODI-SR-B,ODI-WKTS,ODI-SR-BL,RUNS-S,HS,AVE,SR -B,...,COUNTRY_NZ,COUNTRY_PAK,COUNTRY_SA,COUNTRY_SL,COUNTRY_WI,COUNTRY_ZIM,PLAYING ROLE_Batsman,PLAYING ROLE_Bowler,PLAYING ROLE_W. Keeper,CAPTAINCY EXP_1
0,0,0,0,0.00,0,0.0,0,0,0.000000,0.000000,...,0,0,1,0,0,0,0,0,0,0
1,214,18,657,71.41,185,37.6,0,0,0.000000,0.000000,...,0,0,0,0,0,0,0,1,0,0
2,571,58,1269,80.62,288,32.9,167,39,18.555556,121.014493,...,0,0,0,0,0,0,0,1,0,0
3,284,31,241,84.56,51,36.8,58,11,5.800000,76.315789,...,0,0,0,0,0,0,0,1,0,0
4,63,0,79,45.93,0,0.0,1317,71,32.925000,120.714940,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,0,0,0,0.00,0,0.0,49,16,9.800000,125.640000,...,0,0,0,0,0,0,1,0,0,0
126,6398,7,6814,75.78,3,86.6,3,3,3.000000,42.850000,...,0,1,0,0,0,0,1,0,0,1
127,1775,9,8051,87.58,109,44.3,1237,66,26.319149,131.876333,...,0,0,0,0,0,0,1,0,0,1
128,1114,288,790,73.55,278,35.4,99,23,9.900000,91.666667,...,0,0,0,0,0,0,0,1,0,0


In [29]:
Xf=In.columns

In [30]:
X = In[Xf]
# Y = I['SOLD PRICE']

In [31]:
Cor = X.corr().abs()
Cor

,T-RUNS,T-WKTS,ODI-RUNS-S,ODI-SR-B,ODI-WKTS,ODI-SR-BL,RUNS-S,HS,AVE,SR -B,...,COUNTRY_NZ,COUNTRY_PAK,COUNTRY_SA,COUNTRY_SL,COUNTRY_WI,COUNTRY_ZIM,PLAYING ROLE_Batsman,PLAYING ROLE_Bowler,PLAYING ROLE_W. Keeper,CAPTAINCY EXP_1
T-RUNS,1.000000,0.026285,0.892823,0.231411,0.045505,0.067700,0.410143,0.411209,0.374028,0.114301,...,0.078977,0.034201,0.067612,0.104836,0.133967,0.016597,0.384422,0.356923,0.106095,0.690647
T-WKTS,0.026285,1.000000,0.088276,0.012052,0.822940,0.060641,0.218544,0.268432,0.265554,0.147757,...,0.004727,0.016370,0.080644,0.147270,0.011655,0.040595,0.276450,0.405434,0.148530,0.088782
ODI-RUNS-S,0.892823,0.088276,1.000000,0.319264,0.056554,0.126795,0.523955,0.495765,0.446280,0.194111,...,0.078666,0.027848,0.024607,0.138013,0.107451,0.021818,0.367995,0.439249,0.141647,0.714058
ODI-SR-B,0.231411,0.012052,0.319264,1.000000,0.160114,0.284584,0.306338,0.360086,0.340331,0.375368,...,0.109576,0.067246,0.003757,0.098844,0.039412,0.012233,0.083485,0.193205,0.073803,0.291373
ODI-WKTS,0.045505,0.822940,0.056554,0.160114,1.000000,0.124361,0.198101,0.209109,0.222074,0.030409,...,0.020447,0.082970,0.037786,0.257665,0.004124,0.058876,0.358824,0.359333,0.218286,0.077536
ODI-SR-BL,0.067700,0.060641,0.126795,0.284584,0.124361,1.000000,0.027772,0.067615,0.053775,0.051679,...,0.056829,0.016369,0.083771,0.076880,0.060253,0.026320,0.014689,0.066866,0.353423,0.103315
RUNS-S,0.410143,0.218544,0.523955,0.306338,0.198101,0.027772,1.000000,0.834631,0.767009,0.380025,...,0.017446,0.206051,0.015433,0.022329,0.005894,0.069425,0.418555,0.525628,0.239187,0.352422
HS,0.411209,0.268432,0.495765,0.360086,0.209109,0.067615,0.834631,1.000000,0.876252,0.531032,...,0.135315,0.187916,0.010261,0.005006,0.050969,0.078740,0.408826,0.643720,0.266624,0.392749
AVE,0.374028,0.265554,0.446280,0.340331,0.222074,0.053775,0.767009,0.876252,1.000000,0.583579,...,0.047452,0.196173,0.020242,0.041334,0.073533,0.066836,0.432462,0.666092,0.216213,0.378258
SR -B,0.114301,0.147757,0.194111,0.375368,0.030409,0.051679,0.380025,0.531032,0.583579,1.000000,...,0.029239,0.039436,0.093151,0.101379,0.030092,0.020114,0.157905,0.379788,0.155117,0.176920


In [32]:
upper1 = Cor.where(np.triu(np.ones(Cor.shape),k=1).astype(bool))
upper1

,T-RUNS,T-WKTS,ODI-RUNS-S,ODI-SR-B,ODI-WKTS,ODI-SR-BL,RUNS-S,HS,AVE,SR -B,...,COUNTRY_NZ,COUNTRY_PAK,COUNTRY_SA,COUNTRY_SL,COUNTRY_WI,COUNTRY_ZIM,PLAYING ROLE_Batsman,PLAYING ROLE_Bowler,PLAYING ROLE_W. Keeper,CAPTAINCY EXP_1
T-RUNS,NaN,0.026285,0.892823,0.231411,0.045505,0.067700,0.410143,0.411209,0.374028,0.114301,...,0.078977,0.034201,0.067612,0.104836,0.133967,0.016597,0.384422,0.356923,0.106095,0.690647
T-WKTS,NaN,NaN,0.088276,0.012052,0.822940,0.060641,0.218544,0.268432,0.265554,0.147757,...,0.004727,0.016370,0.080644,0.147270,0.011655,0.040595,0.276450,0.405434,0.148530,0.088782
ODI-RUNS-S,NaN,NaN,NaN,0.319264,0.056554,0.126795,0.523955,0.495765,0.446280,0.194111,...,0.078666,0.027848,0.024607,0.138013,0.107451,0.021818,0.367995,0.439249,0.141647,0.714058
ODI-SR-B,NaN,NaN,NaN,NaN,0.160114,0.284584,0.306338,0.360086,0.340331,0.375368,...,0.109576,0.067246,0.003757,0.098844,0.039412,0.012233,0.083485,0.193205,0.073803,0.291373
ODI-WKTS,NaN,NaN,NaN,NaN,NaN,0.124361,0.198101,0.209109,0.222074,0.030409,...,0.020447,0.082970,0.037786,0.257665,0.004124,0.058876,0.358824,0.359333,0.218286,0.077536
ODI-SR-BL,NaN,NaN,NaN,NaN,NaN,NaN,0.027772,0.067615,0.053775,0.051679,...,0.056829,0.016369,0.083771,0.076880,0.060253,0.026320,0.014689,0.066866,0.353423,0.103315
RUNS-S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.834631,0.767009,0.380025,...,0.017446,0.206051,0.015433,0.022329,0.005894,0.069425,0.418555,0.525628,0.239187,0.352422
HS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.876252,0.531032,...,0.135315,0.187916,0.010261,0.005006,0.050969,0.078740,0.408826,0.643720,0.266624,0.392749
AVE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.583579,...,0.047452,0.196173,0.020242,0.041334,0.073533,0.066836,0.432462,0.666092,0.216213,0.378258
SR -B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.029239,0.039436,0.093151,0.101379,0.030092,0.020114,0.157905,0.379788,0.155117,0.176920


In [33]:
Drop1 = [column for column in upper1.columns if any(upper1[column]>0.80)]
Drop1

['ODI-RUNS-S', 'ODI-WKTS', 'HS ', 'AVE', 'SIXERS', 'WKTS', 'SR-BL']

In [34]:
X.drop(Drop1, axis=1, inplace=True)


In [35]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130 entries, 0 to 129
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   T-RUNS                  130 non-null    int64  
 1   T-WKTS                  130 non-null    int64  
 2   ODI-SR-B                130 non-null    float64
 3   ODI-SR-BL               130 non-null    float64
 4   RUNS-S                  130 non-null    int64  
 5   SR -B                   130 non-null    float64
 6   RUNS-C                  130 non-null    int64  
 7   AVE-BL                  130 non-null    float64
 8   ECON                    130 non-null    float64
 9   AGE_2                   130 non-null    uint8  
 10  AGE_3                   130 non-null    uint8  
 11  COUNTRY_BAN             130 non-null    uint8  
 12  COUNTRY_ENG             130 non-null    uint8  
 13  COUNTRY_IND             130 non-null    uint8  
 14  COUNTRY_NZ              130 non-null    ui